In [73]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow.keras import layers
from tqdm import tqdm
import matplotlib.pyplot as plt
from tqdm import tqdm
import foolbox as fb


tf.compat.v1.enable_eager_execution()
tf.keras.backend.clear_session()  # For easy reset of notebook state.

/Users/florianmerkle/dev/foolbox/foolbox/attacks/brendel_bethge.py:781: NumbaDeprecationWarning: The 'numba.jitclass' decorator has moved to 'numba.experimental.jitclass' to better reflect the experimental nature of the functionality. Please update your imports to accommodate this change and see http://numba.pydata.org/numba-doc/latest/reference/deprecation.html#change-of-jitclass-location for the time frame.
  @jitclass(spec=[])
/Users/florianmerkle/dev/foolbox/foolbox/attacks/brendel_bethge.py:1364: NumbaDeprecationWarning: The 'numba.jitclass' decorator has moved to 'numba.experimental.jitclass' to better reflect the experimental nature of the functionality. Please update your imports to accommodate this change and see http://numba.pydata.org/numba-doc/latest/reference/deprecation.html#change-of-jitclass-location for the time frame.
  @jitclass(spec=spec)
/Users/florianmerkle/dev/foolbox/foolbox/attacks/brendel_bethge.py:1560: NumbaDeprecationWarning: The 'numba.jitclass' decorator 

# Load Data

In [77]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

x_train = x_train.reshape(60000, 784).astype('float32') / 255
x_test = x_test.reshape(10000, 784).astype('float32') / 255

x = tf.convert_to_tensor(x_train[:100].reshape(100,28*28))
y = tf.convert_to_tensor([y_train[:100]])[0];

# Define Model

In [26]:
class CustomLayer(layers.Layer):

    def __init__(self, units=32, activation='relu'):
        super(CustomLayer, self).__init__()
        self.units = units
        self.activation = activation

    def build(self, input_shape):
        #print(input_shape)
        self.w = self.add_weight(shape=(input_shape[-1], self.units),
                                 initializer='random_normal',
                                 trainable=True,
                                name='unpruned_weights')
        self.mask = self.add_weight(shape=(self.w.shape),
                                    initializer='ones',
                                    trainable=False,
                                   name='pruning_mask')
        self.pruned_w = self.add_weight(shape=(input_shape[-1], self.units),
                                 initializer='ones',
                                 trainable=False,
                                       name='pruned_weights')
        
    def call(self, inputs):
        #self.mask_2 = tf.multiply(self.mask, self.mask_2)
        self.pruned_w = tf.multiply(self.w, self.mask)
        #print(self.pruned_w.eval())
        x = tf.matmul(inputs, self.pruned_w)
        
        if self.activation == 'relu':
            return tf.keras.activations.relu(x)
        if self.activation == 'softmax':
            return tf.keras.activations.softmax(x)
        raise ValueError('Activation function not implemented')

class LeNet300_100(tf.keras.Model):
    def __init__(self):
        super(LeNet300_100, self).__init__()
        self.dense1 = CustomLayer(300)
        self.dense2 = CustomLayer(100)
        self.dense3 = CustomLayer(10, activation='softmax')
        
    def call(self, inputs):
        
        x = self.dense1(inputs)
        #print(x.shape)
        x = self.dense2(x)
        #print(x.shape)
        return self.dense3(x)
    


# Compile and Train Model

In [64]:
model = LeNet300_100()
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True) ,
              metrics=['accuracy'],
              experimental_run_tf_function=False
              
             )

callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
model.fit(x=x_train,
          y=y_train,
          batch_size=64,
          epochs=2,
          callbacks=[callback],
          validation_data=(x_test, y_test),
         )

Train on 60000 samples, validate on 10000 samples
Epoch 1/2
60000/60000 [==============================] - 4s 70us/sample - loss: 1.5756 - accuracy: 0.8988 - val_loss: 1.5184 - val_accuracy: 0.9460
Epoch 2/2
60000/60000 [==============================] - 4s 66us/sample - loss: 1.5129 - accuracy: 0.9509 - val_loss: 1.5108 - val_accuracy: 0.9523


In [65]:
model.save('./saved-models/mini-pipeline-baseline-model')

INFO:tensorflow:Assets written to: ./saved-models/mini-pipeline-baseline-model/assets


In [66]:
def train_model(model, to_convergence=True):
    if to_convergence == True:
        callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
        model.fit(
            x=x_train,
            y=y_train,
            batch_size=64,
            epochs=100,
            callbacks=[callback],
            validation_data=(x_test, y_test),
            )
    if to_convergence == False:
        model.fit(
            x=x_train,
            y=y_train,
            batch_size=64,
            epochs=2,
            validation_data=(x_test, y_test),
            )
    return model

def prune_weights(model, pruning_ratio):
    weights = model.get_weights()
    weights_to_prune = model.get_weights()
    for index, weight in enumerate(weights):
        if (index == 0) or (index == 3) or (index == 6):
            flat_weights = weight.flatten()
            flat_weights_to_prune = weights_to_prune[index+1].flatten()
            #print (flat_weights_to_prune.shape, flat_weights.shape)
            flat_weights_df = pd.DataFrame(flat_weights)
            flat_weights_to_prune_df = pd.DataFrame(flat_weights_to_prune)
            no_of_weights_to_prune = int(len(flat_weights)*pruning_ratio)
            print(no_of_weights_to_prune)
            indices_to_delete = flat_weights_df.abs().values.argsort(0)[:no_of_weights_to_prune]
            for idx_to_delete in indices_to_delete:
                flat_weights_to_prune[idx_to_delete] = 0
            dims = weights_to_prune[index+1].shape
            weights_reshaped = flat_weights_to_prune.reshape(dims)
            weights_to_prune[index+1] = weights_reshaped
    #print(weights_to_prune)
    return weights_to_prune


In [60]:
weights = model.get_weights()
weights_to_prune = model.get_weights()
weights[2]

array([[1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       ...,
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.]], dtype=float32)

In [62]:
accuracies = []
pruning_ratios = [.0, .3, .5, .7, .8, .9, .95, .97, .98, .99, .995, .999]
for index, pruning_ratio in enumerate(pruning_ratios):
    model = tf.keras.models.load_model('./saved-models/mini-pipeline-baseline-model')
    for i in range(index + 1):
        if i != index:
            pruned_weights = prune_weights(model, pruning_ratios[i])
            model.set_weights(pruned_weights)
            model = train_model(model, to_convergence=False)
        if i == index:
            print('final pruning and eval')
            pruned_weights = prune_weights(model, pruning_ratios[i])
            model.set_weights(pruned_weights)
            model = train_model(model, to_convergence=True)
            accuracies.append(model.evaluate(x_test, y_test, verbose=0))

final pruning and eval
0
0
0
Train on 60000 samples, validate on 10000 samples
Epoch 1/100
60000/60000 [==============================] - 4s 72us/sample - loss: 1.4841 - accuracy: 0.9776 - val_loss: 1.4859 - val_accuracy: 0.9753
Epoch 2/100
60000/60000 [==============================] - 4s 70us/sample - loss: 1.4815 - accuracy: 0.9804 - val_loss: 1.4874 - val_accuracy: 0.9745
Epoch 3/100
60000/60000 [==============================] - 6s 102us/sample - loss: 1.4793 - accuracy: 0.9821 - val_loss: 1.4866 - val_accuracy: 0.9750
Epoch 4/100
60000/60000 [==============================] - 5s 87us/sample - loss: 1.4783 - accuracy: 0.9832 - val_loss: 1.4891 - val_accuracy: 0.9718
0
0
0
Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 5s 81us/sample - loss: 1.4843 - accuracy: 0.9776 - val_loss: 1.4872 - val_accuracy: 0.9746
Epoch 2/5
60000/60000 [==============================] - 4s 71us/sample - loss: 1.4810 - accuracy: 0.9807 - val_loss

60000/60000 [==============================] - 4s 63us/sample - loss: 1.4663 - accuracy: 0.9951 - val_loss: 1.4790 - val_accuracy: 0.9821
Epoch 3/100
60000/60000 [==============================] - 4s 66us/sample - loss: 1.4660 - accuracy: 0.9954 - val_loss: 1.4773 - val_accuracy: 0.9842
Epoch 4/100
60000/60000 [==============================] - 4s 67us/sample - loss: 1.4658 - accuracy: 0.9955 - val_loss: 1.4790 - val_accuracy: 0.9823
Epoch 5/100
60000/60000 [==============================] - 4s 69us/sample - loss: 1.4654 - accuracy: 0.9959 - val_loss: 1.4776 - val_accuracy: 0.9835
Epoch 6/100
60000/60000 [==============================] - 4s 67us/sample - loss: 1.4655 - accuracy: 0.9959 - val_loss: 1.4788 - val_accuracy: 0.9824
0
0
0
Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 4s 74us/sample - loss: 1.4838 - accuracy: 0.9778 - val_loss: 1.4939 - val_accuracy: 0.9678
Epoch 2/5
60000/60000 [==============================] - 4

60000/60000 [==============================] - 4s 60us/sample - loss: 1.4659 - accuracy: 0.9955 - val_loss: 1.4793 - val_accuracy: 0.9816
Epoch 4/5
60000/60000 [==============================] - 5s 81us/sample - loss: 1.4659 - accuracy: 0.9955 - val_loss: 1.4792 - val_accuracy: 0.9821
Epoch 5/5
60000/60000 [==============================] - 5s 88us/sample - loss: 1.4656 - accuracy: 0.9957 - val_loss: 1.4790 - val_accuracy: 0.9819
188160
24000
800
Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 4s 67us/sample - loss: 1.4688 - accuracy: 0.9935 - val_loss: 1.4799 - val_accuracy: 0.9816
Epoch 2/5
60000/60000 [==============================] - 4s 68us/sample - loss: 1.4654 - accuracy: 0.9961 - val_loss: 1.4798 - val_accuracy: 0.9812
Epoch 3/5
60000/60000 [==============================] - 4s 70us/sample - loss: 1.4650 - accuracy: 0.9964 - val_loss: 1.4793 - val_accuracy: 0.9825
Epoch 4/5
60000/60000 [==============================] 

60000/60000 [==============================] - 4s 64us/sample - loss: 1.4804 - accuracy: 0.9855 - val_loss: 1.4828 - val_accuracy: 0.9809
Epoch 2/5
60000/60000 [==============================] - 4s 64us/sample - loss: 1.4700 - accuracy: 0.9935 - val_loss: 1.4816 - val_accuracy: 0.9817
Epoch 3/5
60000/60000 [==============================] - 4s 64us/sample - loss: 1.4679 - accuracy: 0.9950 - val_loss: 1.4812 - val_accuracy: 0.9814
Epoch 4/5
60000/60000 [==============================] - 4s 64us/sample - loss: 1.4665 - accuracy: 0.9958 - val_loss: 1.4806 - val_accuracy: 0.9815
Epoch 5/5
60000/60000 [==============================] - 4s 65us/sample - loss: 1.4658 - accuracy: 0.9962 - val_loss: 1.4802 - val_accuracy: 0.9819
final pruning and eval
223440
28500
950
Train on 60000 samples, validate on 10000 samples
Epoch 1/100
60000/60000 [==============================] - 4s 64us/sample - loss: 1.5054 - accuracy: 0.9640 - val_loss: 1.4953 - val_accuracy: 0.9713
Epoch 2/100
60000/60000 [=====

223440
28500
950
Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 4s 65us/sample - loss: 1.5062 - accuracy: 0.9622 - val_loss: 1.4952 - val_accuracy: 0.9699
Epoch 2/5
60000/60000 [==============================] - 4s 64us/sample - loss: 1.4856 - accuracy: 0.9805 - val_loss: 1.4916 - val_accuracy: 0.9719
Epoch 3/5
60000/60000 [==============================] - 4s 69us/sample - loss: 1.4811 - accuracy: 0.9836 - val_loss: 1.4896 - val_accuracy: 0.9744
Epoch 4/5
60000/60000 [==============================] - 4s 67us/sample - loss: 1.4783 - accuracy: 0.9864 - val_loss: 1.4883 - val_accuracy: 0.9750
Epoch 5/5
60000/60000 [==============================] - 4s 65us/sample - loss: 1.4764 - accuracy: 0.9880 - val_loss: 1.4874 - val_accuracy: 0.9755
final pruning and eval
228144
29100
970
Train on 60000 samples, validate on 10000 samples
Epoch 1/100
60000/60000 [==============================] - 4s 63us/sample - loss: 1.5358 - accuracy: 0.

...

60000/60000 [==============================] - 4s 70us/sample - loss: 1.4717 - accuracy: 0.9896 - val_loss: 1.4815 - val_accuracy: 0.9792
Epoch 5/5
60000/60000 [==============================] - 4s 66us/sample - loss: 1.4712 - accuracy: 0.9902 - val_loss: 1.4812 - val_accuracy: 0.9797
117600
15000
500
Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 4s 64us/sample - loss: 1.4701 - accuracy: 0.9912 - val_loss: 1.4811 - val_accuracy: 0.9800
Epoch 2/5
60000/60000 [==============================] - 4s 63us/sample - loss: 1.4687 - accuracy: 0.9928 - val_loss: 1.4784 - val_accuracy: 0.9829
Epoch 3/5
60000/60000 [==============================] - 4s 63us/sample - loss: 1.4681 - accuracy: 0.9932 - val_loss: 1.4797 - val_accuracy: 0.9822
Epoch 4/5
60000/60000 [==============================] - 4s 64us/sample - loss: 1.4678 - accuracy: 0.9936 - val_loss: 1.4797 - val_accuracy: 0.9813
Epoch 5/5
60000/60000 [==============================] 

60000/60000 [==============================] - 4s 62us/sample - loss: 2.1613 - accuracy: 0.0987 - val_loss: 2.1619 - val_accuracy: 0.0980
Epoch 16/100
60000/60000 [==============================] - 4s 63us/sample - loss: 2.1611 - accuracy: 0.0987 - val_loss: 2.1617 - val_accuracy: 0.0980
Epoch 17/100
60000/60000 [==============================] - 4s 64us/sample - loss: 2.1609 - accuracy: 0.0987 - val_loss: 2.1616 - val_accuracy: 0.0980
Epoch 18/100
60000/60000 [==============================] - 4s 63us/sample - loss: 2.1608 - accuracy: 0.0987 - val_loss: 2.1614 - val_accuracy: 0.0980
Epoch 19/100
60000/60000 [==============================] - 4s 63us/sample - loss: 2.1606 - accuracy: 0.0987 - val_loss: 2.1613 - val_accuracy: 0.0980
Epoch 20/100
60000/60000 [==============================] - 4s 62us/sample - loss: 2.1604 - accuracy: 0.0987 - val_loss: 2.1612 - val_accuracy: 0.0980
Epoch 21/100
60000/60000 [==============================] - 4s 63us/sample - loss: 2.1603 - accuracy: 0.098

60000/60000 [==============================] - 4s 62us/sample - loss: 1.4700 - accuracy: 0.9936 - val_loss: 1.4823 - val_accuracy: 0.9808
Epoch 3/5
60000/60000 [==============================] - 4s 62us/sample - loss: 1.4678 - accuracy: 0.9951 - val_loss: 1.4818 - val_accuracy: 0.9807
Epoch 4/5
60000/60000 [==============================] - 4s 62us/sample - loss: 1.4665 - accuracy: 0.9959 - val_loss: 1.4812 - val_accuracy: 0.9809
Epoch 5/5
60000/60000 [==============================] - 4s 62us/sample - loss: 1.4658 - accuracy: 0.9963 - val_loss: 1.4807 - val_accuracy: 0.9814
223440
28500
950
Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 4s 62us/sample - loss: 1.5153 - accuracy: 0.9544 - val_loss: 1.4949 - val_accuracy: 0.9716
Epoch 2/5
60000/60000 [==============================] - 4s 62us/sample - loss: 1.4851 - accuracy: 0.9808 - val_loss: 1.4914 - val_accuracy: 0.9732
Epoch 3/5
60000/60000 [==============================] 

60000/60000 [==============================] - 4s 63us/sample - loss: 2.2433 - accuracy: 0.0987 - val_loss: 2.2432 - val_accuracy: 0.0980
Epoch 30/100
60000/60000 [==============================] - 4s 63us/sample - loss: 2.2433 - accuracy: 0.0987 - val_loss: 2.2431 - val_accuracy: 0.0980
Epoch 31/100
60000/60000 [==============================] - 4s 62us/sample - loss: 2.2433 - accuracy: 0.0987 - val_loss: 2.2431 - val_accuracy: 0.0980
Epoch 32/100
60000/60000 [==============================] - 4s 63us/sample - loss: 2.2432 - accuracy: 0.0987 - val_loss: 2.2431 - val_accuracy: 0.0980
Epoch 33/100
60000/60000 [==============================] - 4s 62us/sample - loss: 2.2432 - accuracy: 0.0987 - val_loss: 2.2431 - val_accuracy: 0.0980
Epoch 34/100
60000/60000 [==============================] - 4s 63us/sample - loss: 2.2432 - accuracy: 0.0987 - val_loss: 2.2431 - val_accuracy: 0.0980
Epoch 35/100
60000/60000 [==============================] - 4s 63us/sample - loss: 2.2432 - accuracy: 0.098

Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 4s 71us/sample - loss: 1.5331 - accuracy: 0.9409 - val_loss: 1.5093 - val_accuracy: 0.9596
Epoch 2/5
60000/60000 [==============================] - 4s 67us/sample - loss: 1.5010 - accuracy: 0.9669 - val_loss: 1.5027 - val_accuracy: 0.9635
Epoch 3/5
60000/60000 [==============================] - 4s 67us/sample - loss: 1.4952 - accuracy: 0.9711 - val_loss: 1.4990 - val_accuracy: 0.9674
Epoch 4/5
60000/60000 [==============================] - 4s 68us/sample - loss: 1.4916 - accuracy: 0.9743 - val_loss: 1.4969 - val_accuracy: 0.9672
Epoch 5/5
60000/60000 [==============================] - 5s 87us/sample - loss: 1.4890 - accuracy: 0.9764 - val_loss: 1.4956 - val_accuracy: 0.9688
230496
29400
980
Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 5s 77us/sample - loss: 1.5467 - accuracy: 0.9258 - val_loss: 1.5204 - val_accuracy: 0.

In [63]:
accuracies

[[1.489057382965088, 0.9718],
 [1.4818183109283447, 0.9793],
 [1.4802037090301514, 0.981],
 [1.4788380348205565, 0.9824],
 [1.4787117778778076, 0.9823],
 [1.4808717639923095, 0.9804],
 [1.4847409406661987, 0.9757],
 [1.4945303134918213, 0.9676],
 [1.505777965927124, 0.9561],
 [2.1604187896728515, 0.098],
 [2.242934785079956, 0.098],
 [2.3025851249694824, 0.098]]

In [67]:
accuracies2 = []
pruning_ratios = [.0, .3, .5, .7, .8, .9, .95, .97, .98, .99, .995, .999]
for index, pruning_ratio in enumerate(pruning_ratios):
    model = tf.keras.models.load_model('./saved-models/mini-pipeline-baseline-model')
    for i in range(index + 1):
        if i != index:
            pruned_weights = prune_weights(model, pruning_ratios[i])
            model.set_weights(pruned_weights)
            model = train_model(model, to_convergence=False)
        if i == index:
            print('final pruning and eval')
            pruned_weights = prune_weights(model, pruning_ratios[i])
            model.set_weights(pruned_weights)
            model = train_model(model, to_convergence=True)
            accuracies.append(model.evaluate(x_test, y_test, verbose=0))
accuracies2

final pruning and eval
0
0
0
Train on 60000 samples, validate on 10000 samples
Epoch 1/100
60000/60000 [==============================] - 5s 84us/sample - loss: 1.5003 - accuracy: 0.9625 - val_loss: 1.4982 - val_accuracy: 0.9645
Epoch 2/100
60000/60000 [==============================] - 4s 74us/sample - loss: 1.4914 - accuracy: 0.9712 - val_loss: 1.4956 - val_accuracy: 0.9669
Epoch 3/100
60000/60000 [==============================] - 4s 63us/sample - loss: 1.4875 - accuracy: 0.9746 - val_loss: 1.4959 - val_accuracy: 0.9664
Epoch 4/100
60000/60000 [==============================] - 4s 61us/sample - loss: 1.4850 - accuracy: 0.9771 - val_loss: 1.4885 - val_accuracy: 0.9732
Epoch 5/100
60000/60000 [==============================] - 4s 68us/sample - loss: 1.4822 - accuracy: 0.9794 - val_loss: 1.4939 - val_accuracy: 0.9676
Epoch 6/100
60000/60000 [==============================] - 4s 66us/sample - loss: 1.4794 - accuracy: 0.9821 - val_loss: 1.4883 - val_accuracy: 0.9742
Epoch 7/100
60000/600

60000/60000 [==============================] - 4s 60us/sample - loss: 1.4753 - accuracy: 0.9880 - val_loss: 1.4819 - val_accuracy: 0.9800
Epoch 2/100
60000/60000 [==============================] - 4s 62us/sample - loss: 1.4716 - accuracy: 0.9906 - val_loss: 1.4827 - val_accuracy: 0.9790
Epoch 3/100
60000/60000 [==============================] - 4s 61us/sample - loss: 1.4701 - accuracy: 0.9919 - val_loss: 1.4813 - val_accuracy: 0.9805
Epoch 4/100
60000/60000 [==============================] - 4s 62us/sample - loss: 1.4694 - accuracy: 0.9924 - val_loss: 1.4810 - val_accuracy: 0.9803
Epoch 5/100
60000/60000 [==============================] - 4s 61us/sample - loss: 1.4690 - accuracy: 0.9928 - val_loss: 1.4812 - val_accuracy: 0.9803
Epoch 6/100
60000/60000 [==============================] - 4s 61us/sample - loss: 1.4684 - accuracy: 0.9932 - val_loss: 1.4810 - val_accuracy: 0.9806
Epoch 7/100
60000/60000 [==============================] - 4s 61us/sample - loss: 1.4679 - accuracy: 0.9936 - va

0
0
0
Train on 60000 samples, validate on 10000 samples
Epoch 1/2
60000/60000 [==============================] - 4s 71us/sample - loss: 1.5001 - accuracy: 0.9628 - val_loss: 1.4960 - val_accuracy: 0.9655
Epoch 2/2
60000/60000 [==============================] - 4s 65us/sample - loss: 1.4920 - accuracy: 0.9704 - val_loss: 1.4928 - val_accuracy: 0.9691
70560
9000
300
Train on 60000 samples, validate on 10000 samples
Epoch 1/2
60000/60000 [==============================] - 4s 64us/sample - loss: 1.4843 - accuracy: 0.9778 - val_loss: 1.4944 - val_accuracy: 0.9680
Epoch 2/2
60000/60000 [==============================] - 4s 65us/sample - loss: 1.4820 - accuracy: 0.9801 - val_loss: 1.4881 - val_accuracy: 0.9735
117600
15000
500
Train on 60000 samples, validate on 10000 samples
Epoch 1/2
60000/60000 [==============================] - 4s 64us/sample - loss: 1.4781 - accuracy: 0.9840 - val_loss: 1.4856 - val_accuracy: 0.9764
Epoch 2/2
60000/60000 [==============================] - 4s 66us/sample 

60000/60000 [==============================] - 4s 68us/sample - loss: 1.4825 - accuracy: 0.9823 - val_loss: 1.4916 - val_accuracy: 0.9715
Epoch 8/100
60000/60000 [==============================] - 4s 67us/sample - loss: 1.4814 - accuracy: 0.9830 - val_loss: 1.4910 - val_accuracy: 0.9718
Epoch 9/100
60000/60000 [==============================] - 4s 66us/sample - loss: 1.4803 - accuracy: 0.9841 - val_loss: 1.4907 - val_accuracy: 0.9720
Epoch 10/100
60000/60000 [==============================] - 4s 64us/sample - loss: 1.4794 - accuracy: 0.9847 - val_loss: 1.4901 - val_accuracy: 0.9728
Epoch 11/100
60000/60000 [==============================] - 4s 66us/sample - loss: 1.4786 - accuracy: 0.9855 - val_loss: 1.4893 - val_accuracy: 0.9734
Epoch 12/100
60000/60000 [==============================] - 4s 66us/sample - loss: 1.4779 - accuracy: 0.9859 - val_loss: 1.4889 - val_accuracy: 0.9736
Epoch 13/100
60000/60000 [==============================] - 4s 68us/sample - loss: 1.4772 - accuracy: 0.9868 

Epoch 19/100
60000/60000 [==============================] - 4s 73us/sample - loss: 1.4852 - accuracy: 0.9791 - val_loss: 1.4986 - val_accuracy: 0.9647
Epoch 20/100
60000/60000 [==============================] - 4s 69us/sample - loss: 1.4848 - accuracy: 0.9793 - val_loss: 1.4983 - val_accuracy: 0.9645
Epoch 21/100
60000/60000 [==============================] - 4s 74us/sample - loss: 1.4844 - accuracy: 0.9797 - val_loss: 1.4981 - val_accuracy: 0.9651
Epoch 22/100
60000/60000 [==============================] - 4s 67us/sample - loss: 1.4840 - accuracy: 0.9802 - val_loss: 1.4979 - val_accuracy: 0.9645
Epoch 23/100
60000/60000 [==============================] - 4s 74us/sample - loss: 1.4836 - accuracy: 0.9804 - val_loss: 1.4977 - val_accuracy: 0.9652
Epoch 24/100
60000/60000 [==============================] - 4s 67us/sample - loss: 1.4833 - accuracy: 0.9807 - val_loss: 1.4980 - val_accuracy: 0.9645
Epoch 25/100
60000/60000 [==============================] - 4s 70us/sample - loss: 1.4829 - ac

60000/60000 [==============================] - 4s 70us/sample - loss: 2.0952 - accuracy: 0.1896 - val_loss: 2.0959 - val_accuracy: 0.1887
Epoch 26/100
60000/60000 [==============================] - 5s 78us/sample - loss: 2.0951 - accuracy: 0.1896 - val_loss: 2.0957 - val_accuracy: 0.1887
Epoch 27/100
60000/60000 [==============================] - 5s 78us/sample - loss: 2.0950 - accuracy: 0.1895 - val_loss: 2.0959 - val_accuracy: 0.1886
Epoch 28/100
60000/60000 [==============================] - 4s 72us/sample - loss: 2.0949 - accuracy: 0.1896 - val_loss: 2.0960 - val_accuracy: 0.1889
Epoch 29/100
60000/60000 [==============================] - 4s 69us/sample - loss: 2.0949 - accuracy: 0.1896 - val_loss: 2.0958 - val_accuracy: 0.1886
0
0
0
Train on 60000 samples, validate on 10000 samples
Epoch 1/2
60000/60000 [==============================] - 4s 72us/sample - loss: 1.5004 - accuracy: 0.9621 - val_loss: 1.5026 - val_accuracy: 0.9602
Epoch 2/2
60000/60000 [==============================]

Epoch 26/100
60000/60000 [==============================] - 4s 68us/sample - loss: 2.2391 - accuracy: 0.0987 - val_loss: 2.2396 - val_accuracy: 0.0980
Epoch 27/100
60000/60000 [==============================] - 4s 70us/sample - loss: 2.2391 - accuracy: 0.0987 - val_loss: 2.2396 - val_accuracy: 0.0980
Epoch 28/100
60000/60000 [==============================] - 4s 71us/sample - loss: 2.2390 - accuracy: 0.0987 - val_loss: 2.2396 - val_accuracy: 0.0980
0
0
0
Train on 60000 samples, validate on 10000 samples
Epoch 1/2
60000/60000 [==============================] - 4s 69us/sample - loss: 1.5007 - accuracy: 0.9620 - val_loss: 1.4979 - val_accuracy: 0.9640
Epoch 2/2
60000/60000 [==============================] - 4s 62us/sample - loss: 1.4925 - accuracy: 0.9696 - val_loss: 1.4907 - val_accuracy: 0.9713
70560
9000
300
Train on 60000 samples, validate on 10000 samples
Epoch 1/2
60000/60000 [==============================] - 4s 62us/sample - loss: 1.4845 - accuracy: 0.9779 - val_loss: 1.4880 - val

60000/60000 [==============================] - 4s 68us/sample - loss: 2.2964 - accuracy: 0.0987 - val_loss: 2.2966 - val_accuracy: 0.0980
Epoch 26/100
60000/60000 [==============================] - 4s 66us/sample - loss: 2.2964 - accuracy: 0.0987 - val_loss: 2.2966 - val_accuracy: 0.0980
Epoch 27/100
60000/60000 [==============================] - 4s 66us/sample - loss: 2.2964 - accuracy: 0.0987 - val_loss: 2.2966 - val_accuracy: 0.0980
Epoch 28/100
60000/60000 [==============================] - 4s 64us/sample - loss: 2.2963 - accuracy: 0.0987 - val_loss: 2.2966 - val_accuracy: 0.0980
Epoch 29/100
60000/60000 [==============================] - 4s 65us/sample - loss: 2.2963 - accuracy: 0.0987 - val_loss: 2.2966 - val_accuracy: 0.0980
Epoch 30/100
60000/60000 [==============================] - 4s 68us/sample - loss: 2.2963 - accuracy: 0.0987 - val_loss: 2.2966 - val_accuracy: 0.0980
Epoch 31/100
60000/60000 [==============================] - 4s 64us/sample - loss: 2.2963 - accuracy: 0.098

[]

In [71]:
accuracies

[[1.489057382965088, 0.9718],
 [1.4818183109283447, 0.9793],
 [1.4802037090301514, 0.981],
 [1.4788380348205565, 0.9824],
 [1.4787117778778076, 0.9823],
 [1.4808717639923095, 0.9804],
 [1.4847409406661987, 0.9757],
 [1.4945303134918213, 0.9676],
 [1.505777965927124, 0.9561],
 [2.1604187896728515, 0.098],
 [2.242934785079956, 0.098],
 [2.3025851249694824, 0.098],
 [1.485667152404785, 0.9755],
 [1.4832549690246581, 0.9778],
 [1.4841881740570069, 0.9775],
 [1.4811275800704955, 0.9797],
 [1.4808873607635498, 0.9801],
 [1.48083856010437, 0.9805],
 [1.4839636331558228, 0.9783],
 [1.4880072204589845, 0.9737],
 [1.4976928323745728, 0.9653],
 [2.095776139450073, 0.1886],
 [2.2396354637145994, 0.098],
 [2.2965633655548094, 0.098]]

In [94]:
all_success_rates = []
all_accuracies = []
for j in range(10):
    accuracies = []
    success_rates = []
    pruning_ratios = [.0, .3, .5, .7, .8, .9, .95, .97, .98, .99]
    for index, pruning_ratio in tqdm(enumerate(pruning_ratios)):
        model = tf.keras.models.load_model('./saved-models/mini-pipeline-baseline-model')
        for i in range(index + 1):
            if i != index:
                pruned_weights = prune_weights(model, pruning_ratios[i])
                model.set_weights(pruned_weights)
                model = train_model(model, to_convergence=False)
            if i == index:
                print('final pruning and eval')
                pruned_weights = prune_weights(model, pruning_ratios[i])
                model.set_weights(pruned_weights)
                model = train_model(model, to_convergence=True)
                accuracies.append(model.evaluate(x_test, y_test, verbose=0))
                success_rates.append(fgsm_attack(model))
    all_accuracies.append(accuracies)
    all_success_rates.append(success_rates)


0it [00:00, ?it/s]

final pruning and eval
0
0
0
Train on 60000 samples, validate on 10000 samples
Epoch 1/100
60000/60000 [==============================] - 5s 85us/sample - loss: 1.5002 - accuracy: 0.9629 - val_loss: 1.5034 - val_accuracy: 0.9598
Epoch 2/100
60000/60000 [==============================] - 5s 78us/sample - loss: 1.4930 - accuracy: 0.9692 - val_loss: 1.4936 - val_accuracy: 0.9683
Epoch 3/100
60000/60000 [==============================] - 5s 77us/sample - loss: 1.4879 - accuracy: 0.9741 - val_loss: 1.4905 - val_accuracy: 0.9714
Epoch 4/100
60000/60000 [==============================] - 4s 74us/sample - loss: 1.4834 - accuracy: 0.9787 - val_loss: 1.4903 - val_accuracy: 0.9712
Epoch 5/100
60000/60000 [==============================] - 5s 77us/sample - loss: 1.4813 - accuracy: 0.9804 - val_loss: 1.4862 - val_accuracy: 0.9756
Epoch 6/100
60000/60000 [==============================] - 4s 72us/sample - loss: 1.4795 - accuracy: 0.9824 - val_loss: 1.4915 - val_accuracy: 0.9695
Epoch 7/100
60000/600

1it [00:37, 37.80s/it]

0
0
0
Train on 60000 samples, validate on 10000 samples
Epoch 1/2
60000/60000 [==============================] - 5s 79us/sample - loss: 1.4999 - accuracy: 0.9629 - val_loss: 1.4981 - val_accuracy: 0.9647
Epoch 2/2
60000/60000 [==============================] - 5s 75us/sample - loss: 1.4924 - accuracy: 0.9703 - val_loss: 1.4947 - val_accuracy: 0.9665
final pruning and eval
70560
9000
300
Train on 60000 samples, validate on 10000 samples
Epoch 1/100
60000/60000 [==============================] - 4s 72us/sample - loss: 1.4848 - accuracy: 0.9773 - val_loss: 1.4906 - val_accuracy: 0.9710
Epoch 2/100
60000/60000 [==============================] - 4s 74us/sample - loss: 1.4810 - accuracy: 0.9811 - val_loss: 1.4869 - val_accuracy: 0.9746
Epoch 3/100
60000/60000 [==============================] - 4s 72us/sample - loss: 1.4794 - accuracy: 0.9826 - val_loss: 1.4859 - val_accuracy: 0.9758
Epoch 4/100
60000/60000 [==============================] - 4s 74us/sample - loss: 1.4777 - accuracy: 0.9840 - 

2it [01:52, 48.95s/it]

...

0
0
0
Train on 60000 samples, validate on 10000 samples
Epoch 1/2
60000/60000 [==============================] - 4s 65us/sample - loss: 1.5008 - accuracy: 0.9615 - val_loss: 1.4993 - val_accuracy: 0.9636
Epoch 2/2
60000/60000 [==============================] - 4s 60us/sample - loss: 1.4927 - accuracy: 0.9695 - val_loss: 1.4945 - val_accuracy: 0.9674
70560
9000
300
Train on 60000 samples, validate on 10000 samples
Epoch 1/2
60000/60000 [==============================] - 4s 60us/sample - loss: 1.4849 - accuracy: 0.9772 - val_loss: 1.4881 - val_accuracy: 0.9737
Epoch 2/2
60000/60000 [==============================] - 4s 59us/sample - loss: 1.4817 - accuracy: 0.9806 - val_loss: 1.4885 - val_accuracy: 0.9736
117600
15000
500
Train on 60000 samples, validate on 10000 samples
Epoch 1/2
60000/60000 [==============================] - 4s 59us/sample - loss: 1.4777 - accuracy: 0.9845 - val_loss: 1.4847 - val_accuracy: 0.9765
Epoch 2/2
60000/60000 [==============================] - 4s 60us/sample 

9it [11:47, 105.62s/it]

0
0
0
Train on 60000 samples, validate on 10000 samples
Epoch 1/2
60000/60000 [==============================] - 4s 67us/sample - loss: 1.5003 - accuracy: 0.9621 - val_loss: 1.4955 - val_accuracy: 0.9674
Epoch 2/2
60000/60000 [==============================] - 4s 61us/sample - loss: 1.4923 - accuracy: 0.9700 - val_loss: 1.4928 - val_accuracy: 0.9693
70560
9000
300
Train on 60000 samples, validate on 10000 samples
Epoch 1/2
60000/60000 [==============================] - 4s 58us/sample - loss: 1.4857 - accuracy: 0.9764 - val_loss: 1.4894 - val_accuracy: 0.9731
Epoch 2/2
60000/60000 [==============================] - 4s 60us/sample - loss: 1.4816 - accuracy: 0.9804 - val_loss: 1.4851 - val_accuracy: 0.9764
117600
15000
500
Train on 60000 samples, validate on 10000 samples
Epoch 1/2
60000/60000 [==============================] - 4s 59us/sample - loss: 1.4777 - accuracy: 0.9845 - val_loss: 1.4835 - val_accuracy: 0.9783
Epoch 2/2
60000/60000 [==============================] - 4s 59us/sample 

60000/60000 [==============================] - 4s 60us/sample - loss: 2.0817 - accuracy: 0.3811 - val_loss: 2.0851 - val_accuracy: 0.3850
Epoch 34/100
60000/60000 [==============================] - 4s 61us/sample - loss: 2.0816 - accuracy: 0.3816 - val_loss: 2.0852 - val_accuracy: 0.3848


10it [14:59, 89.95s/it] 


In [69]:
def fgsm_attack(model_to_attack):
    fmodel = fb.models.TensorFlowModel(model_to_attack, bounds=(0,1))
    attack = fb.attacks.FGSM()
    adversarials = attack(
        fmodel,
        x,
        y,
        epsilons=[15/255]
    )
    return np.count_nonzero(adversarials[2])/100

In [101]:
(success_rates)

[0.32, 0.32, 0.36, 0.35, 0.3, 0.27, 0.33, 0.49, 0.44, 0.7]

In [89]:
diese = [0]

In [90]:
das = [1]

In [91]:
diese.append(das)

In [92]:
diese

[0, [1]]